In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import time

In [2]:
utilities_df = pd.read_csv('../Resources/Data/Clean_Utilities_zipcode.csv')
utilities_df

,Unnamed: 0,Utility name,City,People served,Zip
0,0,City of Pendleton,"Pendleton, OR","Population served:17,310",97810
1,1,City of Hermiston,"Hermiston, OR","Population served:18,200",97810
2,2,"Milton-freewater, City Of","Milton-freewater, OR","Population served:7,070",97810
3,3,"Umatilla, City of","Umatilla, OR","Population served:6,980",97810
4,4,"Stanfield, City of","Stanfield, OR","Population served:2,115",97810
...,...,...,...,...,...
413,15,Quail Valley Park Improvement District,Prineville,Population served: 110,97752
414,16,Shoun Crossroads,Prineville,Population served: 41,97752
415,17,Sunset Hills Domestic Water Association,Prineville,Population served: 34,97752
416,18,Terrace Mobile Plaza,Prineville,Population served: 144,97752


In [5]:
utilities_df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
utilities_df

,Utility name,City,People served,Zip
0,City of Pendleton,"Pendleton, OR","Population served:17,310",97810
1,City of Hermiston,"Hermiston, OR","Population served:18,200",97810
2,"Milton-freewater, City Of","Milton-freewater, OR","Population served:7,070",97810
3,"Umatilla, City of","Umatilla, OR","Population served:6,980",97810
4,"Stanfield, City of","Stanfield, OR","Population served:2,115",97810
...,...,...,...,...
413,Quail Valley Park Improvement District,Prineville,Population served: 110,97752
414,Shoun Crossroads,Prineville,Population served: 41,97752
415,Sunset Hills Domestic Water Association,Prineville,Population served: 34,97752
416,Terrace Mobile Plaza,Prineville,Population served: 144,97752


In [8]:
utility_list = utilities_df['Utility name'].to_list()
utility_list

['City of Pendleton',
 'City of Hermiston',
 'Milton-freewater, City Of',
 'Umatilla, City of',
 'Stanfield, City of',
 'Adrian, City of',
 'City of Ontario',
 'Fischers Place Mobile Home Park',
 'Jordan Valley, City of',
 'Nyssa, City of',
 'River Village Mobile Home Park',
 'Vale, City of',
 'Brookings, City of',
 'Gold Beach, City of',
 "Harbor Water People's Utility District",
 'Langlois Water District',
 'Nesika Beach-ophir Water District',
 'Old Sheep Ranch Water Association',
 'Port Orford, City of',
 'Rainbow Rock Service Association',
 'Rainbow Rock Village Mobile Home Park',
 'Saunders Creek Hoc',
 'City of Corvallis',
 'City of Albany',
 'City of Lebanon',
 'City of Dallas',
 'Monmouth, City of',
 'Sweet Home, City of',
 'Independence Water System',
 'Philomath Public Works',
 'Harrisburg, City of',
 'Luckiamute Domestic Water Co-op',
 'Lyons Mehama Water District',
 'Bandon, City of',
 'Bridge Water District',
 'Coos Bay North Bend Water Board',
 'Coquille, City of',
 'Coun

In [40]:
driver = webdriver.Chrome()
driver.get("https://www.ewg.org/tapwater/advanced-search.php")

In [35]:
utility_list.index('City Of Alturas')

397

In [38]:
some_utils = utility_list[398:]

In [37]:
some_utils

['City Of Alturas',
 "I'sot Well #3 & #15",
 'Newell County Water District',
 'Avion Water Company - Brasada Ranch',
 'Avion Water Company - Red Cloud',
 'Barnes Butte Homeowners Association',
 'Cascade Pines Mobile Home Park',
 'Dry Creek Airpark',
 'Happy Hollow Water Company',
 'High Desert Estates',
 'Highland Subdivision Water District',
 'Idleway Improvement District',
 'Mcdougal Water System',
 'Ochoco Valley Home Improvement District',
 'Ochoco West Water & San Authority',
 'Prineville Mobile Home Park',
 'Quail Valley Park Improvement District',
 'Shoun Crossroads',
 'Sunset Hills Domestic Water Association',
 'Terrace Mobile Plaza',
 'Westridge Subdivision']

In [44]:
#contaminant_list = []
for utility in some_utils:
    try:
        driver.get("https://www.ewg.org/tapwater/advanced-search.php")
        time.sleep(2)
        utility_input = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[1]')
        utility_input.clear()
        utility_input.send_keys(utility)
        go_btn = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[3]')
        go_btn.click()
        time.sleep(2)
        utility = driver.find_element(By.XPATH, '/html/body/div[3]/main/figure/table/tbody/tr/td[1]/a')
        utility.click()

        page_source = driver.page_source

        html_soup = soup(page_source, 'html.parser')

        #Get the name of the Water Utility
        Utility = html_soup.find('h1').text
        Utility


        #get the html data we need
        data_box = html_soup.find_all('div', class_='contaminant-name')

        for i in range(len(data_box)):
            data = data_box[i].find_all('span')
            data_measure = []
            d = {
                'Utility' : Utility,
                'Contaminant': '', 
                'Utility Measuremnt':'', 
                'EWG HEALTH GUIDELINE': '',
                'Legal Limit':'' 
            }

            contaminant_name = data_box[i].find('h3')
            d['Contaminant'] = contaminant_name

            for j in range(len(data)):
                measurement = data[j].text
                #print(measurement)
                data_measure.append(measurement)
                #print(data_measure)

            try:
                d['Utility Measuremnt'] = data_measure[data_measure.index('THIS UTILITY')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong")
            try:
                d['EWG HEALTH GUIDELINE'] = data_measure[data_measure.index('EWG HEALTH GUIDELINE')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong") 
            try:
                d['Legal Limit'] = data_measure[data_measure.index('LEGAL LIMIT')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong") 

            contaminant_list.append(d)
    except:
        pass

A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose


In [45]:
contaminant_list

[{'Utility': 'City of Pendleton',
  'Contaminant': <h3>Arsenic</h3>,
  'Utility Measuremnt': '0.750 ppb',
  'EWG HEALTH GUIDELINE': '0.004 ppb',
  'Legal Limit': '10 ppb'},
 {'Utility': 'City of Pendleton',
  'Contaminant': <h3>Chromium (hexavalent)</h3>,
  'Utility Measuremnt': '0.0472 ppb',
  'EWG HEALTH GUIDELINE': '0.02 ppb',
  'Legal Limit': ''},
 {'Utility': 'City of Pendleton',
  'Contaminant': <h3>Haloacetic acids (HAA5)†</h3>,
  'Utility Measuremnt': '15.3 ppb',
  'EWG HEALTH GUIDELINE': '0.1 ppb',
  'Legal Limit': '60 ppb'},
 {'Utility': 'City of Pendleton',
  'Contaminant': <h3>Haloacetic acids (HAA9)†</h3>,
  'Utility Measuremnt': '16.2 ppb',
  'EWG HEALTH GUIDELINE': '0.06 ppb',
  'Legal Limit': ''},
 {'Utility': 'City of Pendleton',
  'Contaminant': <h3>Nitrate</h3>,
  'Utility Measuremnt': '1.40 ppm',
  'EWG HEALTH GUIDELINE': '0.14 ppm',
  'Legal Limit': '10 ppm'},
 {'Utility': 'City of Pendleton',
  'Contaminant': <h3>Total trihalomethanes (TTHMs)†</h3>,
  'Utility Mea

In [46]:
df = pd.DataFrame(contaminant_list)

In [47]:
df

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,City of Pendleton,[Arsenic],0.750 ppb,0.004 ppb,10 ppb
1,City of Pendleton,[Chromium (hexavalent)],0.0472 ppb,0.02 ppb,
2,City of Pendleton,[Haloacetic acids (HAA5)†],15.3 ppb,0.1 ppb,60 ppb
3,City of Pendleton,[Haloacetic acids (HAA9)†],16.2 ppb,0.06 ppb,
4,City of Pendleton,[Nitrate],1.40 ppm,0.14 ppm,10 ppm
...,...,...,...,...,...
2999,Westridge Subdivision,"[Radium, combined (-226 & -228)]",0.76 pCi/L,0.05 pCi/L,5 pCi/L
3000,Westridge Subdivision,[Dichloromethane (methylene chloride)],0.220 ppb,4 ppb,5 ppb
3001,Westridge Subdivision,[Fluoride],0.235 ppm,,4 ppm
3002,Westridge Subdivision,[Manganese],13.5 ppb,100 ppb,


In [48]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

In [49]:
df = df.astype({"Contaminant": str})

In [50]:
df.Contaminant = df.Contaminant.apply(lambda x: cleanhtml(x))

In [51]:
df

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,City of Pendleton,Arsenic,0.750 ppb,0.004 ppb,10 ppb
1,City of Pendleton,Chromium (hexavalent),0.0472 ppb,0.02 ppb,
2,City of Pendleton,Haloacetic acids (HAA5)†,15.3 ppb,0.1 ppb,60 ppb
3,City of Pendleton,Haloacetic acids (HAA9)†,16.2 ppb,0.06 ppb,
4,City of Pendleton,Nitrate,1.40 ppm,0.14 ppm,10 ppm
...,...,...,...,...,...
2999,Westridge Subdivision,"Radium, combined (-226 &amp; -228)",0.76 pCi/L,0.05 pCi/L,5 pCi/L
3000,Westridge Subdivision,Dichloromethane (methylene chloride),0.220 ppb,4 ppb,5 ppb
3001,Westridge Subdivision,Fluoride,0.235 ppm,,4 ppm
3002,Westridge Subdivision,Manganese,13.5 ppb,100 ppb,


In [52]:
df.to_csv('../Resources/Data/Raw_OR_EWG_Scrape_v1.csv')

In [53]:
driver.quit()